`pandas`包含的数据结构和数据处理工具的设计使得在Python中进行数据清洗和分析非常快捷

`pandas`经常是和其他数值计算工具，比如`NumPy`和`scrPy`,以及数据可视化工具`matplotlib`一起使用的

`pandas`支持大部分NumPy语言风格的数组计算，尤其是**数组函数以及没有for循环的各种数据处理**

但是**最大的不同**是：
- `pandas`用来处理**表格数据或异质数据**
- `NumPy`更适合**处理同质性的数值类数组数据**

In [1]:
import pandas as pd
from pandas import Series, DataFrame # 最常用的类

In [3]:
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.set_printoptions(precision=4, suppress=True) # numpy 不要使用科学计数法，小数4位

## pandas数据结构介绍

`pandas`提供了两个常用的工具**数据结构**:`Series`和`DataFrame`

### Series

`Series`: 是 一种**一维的数组对象**，包含了**一个值序列**(与NumPy中的类型相似)，并且包含了**数据标签**，称为**索引** `index`
- 一般字符串表示 **索引在左侧，值在右边，分两列显示**
- `index`默认从`0-N-1`（N值的长度）
- `values`属性： 获取Series的值
- `index`属性：  获取Series的索引

通常需要创建一个索引序列，**用标签标识每个数据点**

In [4]:
obj = Series([4, 5, -1,5,6])
obj

0    4
1    5
2   -1
3    5
4    6
dtype: int64

In [5]:
obj.values

array([ 4,  5, -1,  5,  6])

In [6]:
obj.index # 默认索引

RangeIndex(start=0, stop=5, step=1)

In [7]:
type(obj.values)

numpy.ndarray

In [9]:
type(obj.index) # 默认索引

pandas.core.indexes.range.RangeIndex

In [10]:
obj2 = pd.Series([4,7,-3,-5], index=['d', 'b', 'c', 'a'])
obj2

d    4
b    7
c   -3
a   -5
dtype: int64

In [11]:
type(obj2.values),type(obj2.index)

(numpy.ndarray, pandas.core.indexes.base.Index)

In [13]:
obj2.index

Index(['d', 'b', 'c', 'a'], dtype='object')

In [12]:
obj3 = pd.Series([4,7,-3,-5], index=['d', 'b'])
obj3

ValueError: Length of passed values is 4, index implies 2.

😀 `Series`数据结构解析
 - `values`: 类型是`numpy.ndarray`, **值是数组**
 - `index`: 默认索引是`pandas.core.indexes.range.RangeIndex`， 类似range对象
     - 自定义索引是`pandas.core.indexes.base.Index`,`pandas.core.indexes.numeric.Int64Index`等，**是`pandas.core.indexes`实例**
     - 都是`Index`类型
- 指定`index`的时候，**`index`和`values`的长度必须一致** `ValueError: Length of passed values is 4, index implies 2`

In [14]:
obj2

d    4
b    7
c   -3
a   -5
dtype: int64

In [16]:
obj2['d'], obj2[0]

(4, 4)

In [17]:
obj2['d'] = 12
obj2[1] = 22
obj2

d    12
b    22
c    -3
a    -5
dtype: int64

In [18]:
obj2 > 0

d     True
b     True
c    False
a    False
dtype: bool

In [19]:
obj2[obj2 > 0]  #bool索引

d    12
b    22
dtype: int64

In [20]:
obj3 = Series([11,12,13,0,-11], index=[1,2,3,4,5])
obj3

1    11
2    12
3    13
4     0
5   -11
dtype: int64

In [21]:
obj3.index, type(obj3.index)

(Int64Index([1, 2, 3, 4, 5], dtype='int64'),
 pandas.core.indexes.numeric.Int64Index)

In [22]:
obj3[1]

11

🐖 **创建自动以的索引标签时，默认索引就不会存在了**
- 但是实际使用中，依然可以使用`obj[1]`获取对应的数值，**是因为 `Index`依然有索引，实际上是先在`index`查找索引标签，在从索引返回对象的数**

🐖 可以使用标签来进行数据的选择
- 尤其是 **索引列表**， 可以选取对应Series子集  ` key of type tuple not found and not a MultiIndex`

In [29]:
obj2

d    12
b    22
c    -3
a    -5
dtype: int64

In [30]:
obj2['a']

-5

In [31]:
obj2['a', 'b']

ValueError: key of type tuple not found and not a MultiIndex

In [32]:
obj2[['a', 'b']]

a    -5
b    22
dtype: int64

😀 使用NumPy的函数或者NumPy风格的操作，比如使用布尔值进行过滤，与标量相乘等，这些操作将 **保存索引值连接**

In [33]:
obj2 * 2

d    24
b    44
c    -6
a   -10
dtype: int64

In [34]:
obj2 > 0

d     True
b     True
c    False
a    False
dtype: bool

In [35]:
obj2[ obj2 > 0 ]

d    12
b    22
dtype: int64

从字典角度考虑Series

- 从上面的例子中看出，可以将Series看做**是从`index`到`value`的映射**, **索引值和数据值按位置配对**
- 看做是 **一个长度固定且有序的字典**
- 操作使用`[]`,**传入键获取值**
- `in` 操作判断的是 **`Index`是是否存在**

In [42]:
'a' in obj2

True

In [43]:
1 in obj2

False

**在可以使用字典的上下文中，也可以使用Series**

In [36]:
isinstance(obj2, dict)

False

👉 **从字典构建Series**

In [37]:
sdata = {'Ohio': 35000, 'Texas': 71000, 'Oregon': 16000 , 'Utah': 5000}
sdata

{'Ohio': 35000, 'Texas': 71000, 'Oregon': 16000, 'Utah': 5000}

In [39]:
obj3 = pd.Series(sdata) # 有序且长度固定的字典
obj3

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

ლ(′◉❥◉｀ლ)  当把字传递给Series构造函数时，产生的Series的**索引值将是排序好的字典键**

可以**将字典键按照想要的顺序传递给构造函数**，从而产生的Series的索引符合预期
- **按照传入的`index`的顺序从字典中获取`value`，构造Series**
- 如果字典中的键不在index中，就被排除在外了
- 如果index中的键**没有在字典中找到**， 使用`NAN`（`not a number`）填充
- `NAN`是pandas标记**缺失值或者NA值**的方式
    - `isnull`,`notnull`检查缺失值
    -  任何值与`NA`计算结果依然是`NA`

In [41]:
obj4 = pd.Series(sdata, index=['California','Ohio','Oregon',  'Texas' ])
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

In [46]:
obj4.isnull()

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

In [48]:
obj4.notnull()

California    False
Ohio           True
Oregon         True
Texas          True
dtype: bool

在数学操作中，**自动对齐索引**是Series中一个非常有用的特性; 和数据库中的`join`操作类似

In [49]:
obj3 + obj4

California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64

`Series`对象自身和其索引都有`name`属性

In [50]:
obj4.name = 'population'
obj4.index.name = 'state'
obj4 

state
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
Name: population, dtype: float64

`Series`索引可以通过**按位赋值**的方式进行修改

In [51]:
obj

0    4
1    5
2   -1
3    5
4    6
dtype: int64

In [52]:
obj.index = ['Bob', 'Steve', 'Jeff', 'Bob', 'Steve']
obj 

Bob      4
Steve    5
Jeff    -1
Bob      5
Steve    6
dtype: int64

In [53]:
obj['Bob']

Bob    4
Bob    5
dtype: int64

In [56]:
type(obj['Jeff'])

numpy.int64

In [55]:
type(obj['Bob'])

pandas.core.series.Series

### DataFrame

`DataFrame`: 表示的**矩阵的数据表**，表格型数据

- 包含已排序的**列集合**
- 每一列可以是**不同的值类型**(数值，字符串，布尔型)
- **既有行索引，也有列索引**
- 被视为： **一个共享相同索引的Series的字典**

`DataFrame`中，数据被存储为一个**二维块**，而不是列表，字典或者其他的一维数组的集合

从`DataFrame`选取的列是数据的**视图**，而不是拷贝，对Series的修改会映射到`DataFrame`

① 利用 **包含等长度列表或NumPy数组的字典**来生成DataFrame

In [57]:
data = {
    
    'state':['Ohio', 'Ohio', 'Ohio' ,'Nevada', 'Nevada','Nevada' ],
    'year':[2000,2001,2002,2001,2002,2003],
    'pop':[1.5,1.7,3.6,2.4,2.9,3.2]
    
}

In [68]:
frame = DataFrame(data)
frame

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9
5,Nevada,2003,3.2


In [60]:
type(frame.values)

numpy.ndarray

In [62]:
type(frame.columns), type(frame.index)

(pandas.core.indexes.base.Index, pandas.core.indexes.range.RangeIndex)

In [69]:
DataFrame(data, columns=['year', 'pop', 'state'])

,year,pop,state
0,2000,1.5,Ohio
1,2001,1.7,Ohio
2,2002,3.6,Ohio
3,2001,2.4,Nevada
4,2002,2.9,Nevada
5,2003,3.2,Nevada


In [71]:
frame2 = DataFrame(data, columns=['year', 'state', 'pop', 'debt'],
                  index=['one', 'two', 'three','four', 'five','six'])
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,NaN
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,NaN
five,2002,Nevada,2.9,NaN
six,2003,Nevada,3.2,NaN


In [73]:
type(frame2.debt)

pandas.core.series.Series

In [74]:
frame2.debt = 1000
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,1000
two,2001,Ohio,1.7,1000
three,2002,Ohio,3.6,1000
four,2001,Nevada,2.4,1000
five,2002,Nevada,2.9,1000
six,2003,Nevada,3.2,1000


In [76]:
frame.state

0      Ohio
1      Ohio
2      Ohio
3    Nevada
4    Nevada
5    Nevada
Name: state, dtype: object

In [77]:
frame2.state

one        Ohio
two        Ohio
three      Ohio
four     Nevada
five     Nevada
six      Nevada
Name: state, dtype: object

In [75]:
frame2.columns

Index(['year', 'state', 'pop', 'debt'], dtype='object')

In [78]:
frame2.loc

In [79]:
frame2.iloc

In [80]:
frame2.loc['one']

year     2000
state    Ohio
pop       1.5
debt     1000
Name: one, dtype: object

In [82]:
frame2.iloc[1]

year     2001
state    Ohio
pop       1.7
debt     1000
Name: two, dtype: object

In [83]:
type(frame2.loc['one'])

pandas.core.series.Series

`loc`和`iloc`属性将`DataFrame`转换为**行提取**，返回的类型是`pandas.core.indexing._iLocIndexer`,返回的依然是`Series`

In [86]:
frame2.debt = np.arange(6.)
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,0.0
two,2001,Ohio,1.7,1.0
three,2002,Ohio,3.6,2.0
four,2001,Nevada,2.4,3.0
five,2002,Nevada,2.9,4.0
six,2003,Nevada,3.2,5.0


In [87]:
frame2.debt = Series([11,2,23,4,5,56])
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,NaN
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,NaN
five,2002,Nevada,2.9,NaN
six,2003,Nevada,3.2,NaN


赋值：
- 标量： 进行扩展，赋值
- 列表/数组： 长度必须相同
- Series： 自动索引对齐 ，NA

In [93]:
frame2['estern'] = (frame2['pop'] > 3 )
frame2

,year,state,pop,debt,estern
one,2000,Ohio,1.5,NaN,False
two,2001,Ohio,1.7,NaN,False
three,2002,Ohio,3.6,NaN,True
four,2001,Nevada,2.4,NaN,False
five,2002,Nevada,2.9,NaN,False
six,2003,Nevada,3.2,NaN,True


In [96]:
del frame2['estern']

In [97]:
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,NaN
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,NaN
five,2002,Nevada,2.9,NaN
six,2003,Nevada,3.2,NaN


②使用**包含字典的嵌套字典**来创建DataFrame
- 将字典的键作为列
- 内部字典的键作为行索引

In [100]:
pop = {
    
    'Nevada': {2001: 2.4, 2002: 2.9 },
    'Ohio': {2000: 1.5, 2001: 1.7, 2002:3.6}
}

In [111]:
frame3 = DataFrame(pop)
frame3

,Nevada,Ohio
2001,2.4,1.7
2002,2.9,3.6
2000,NaN,1.5


In [112]:
frame3.T

,2001,2002,2000
Nevada,2.4,2.9,NaN
Ohio,1.7,3.6,1.5


In [113]:
DataFrame(pop, index=[2000, 2001,2002, 2003])

,Nevada,Ohio
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6
2003,NaN,NaN


③ 由Series构成的字典创建DataFrame

In [114]:
pdata = {
    'Ohio': frame3['Ohio'][:-1],
     'Nevada': frame3['Nevada'][:2],
}

In [115]:
pdata

{'Ohio': 2001    1.7
 2002    3.6
 Name: Ohio, dtype: float64,
 'Nevada': 2001    2.4
 2002    2.9
 Name: Nevada, dtype: float64}

In [117]:
DataFrame(pdata)

,Ohio,Nevada
2001,1.7,2.4
2002,3.6,2.9


In [118]:
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,NaN
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,NaN
five,2002,Nevada,2.9,NaN
six,2003,Nevada,3.2,NaN


In [119]:
frame2.name = 'Population'
frame2.index.name  = 'ID'
frame2.columns.name = 'Co'

In [120]:
frame2

Co,year,state,pop,debt
ID,,,,
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,NaN
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,NaN
five,2002,Nevada,2.9,NaN
six,2003,Nevada,3.2,NaN


In [121]:
frame2.values

array([[2000, 'Ohio', 1.5, nan],
       [2001, 'Ohio', 1.7, nan],
       [2002, 'Ohio', 3.6, nan],
       [2001, 'Nevada', 2.4, nan],
       [2002, 'Nevada', 2.9, nan],
       [2003, 'Nevada', 3.2, nan]], dtype=object)

### 索引对象

`pandas`中的索引对象**用于存储轴标签和其他元数据**(轴名称或者标签)
- 构造Series或者DataFrame时，所使用的任意数组或标签序列都可以在内部转化为索引对象

o(￣▽￣)ｄ **索引对象是不可变的**，因此用户无法修改索引对象

In [122]:
labels  =pd.Index(np.arange(3))
labels

Int64Index([0, 1, 2], dtype='int64')

In [123]:
labels[1] = 10 # Index does not support mutable operations

TypeError: Index does not support mutable operations

In [124]:
obj2 = Series([0,1,2], index=labels)

In [125]:
obj2

0    0
1    1
2    2
dtype: int64

In [126]:
obj2.index is labels

True

**索引对象像一个固定大小的集合**， 支持一些集合逻辑操作

**索引可以包含重复的标签**，根据重复标签进行栓选，会选取所有重复标签对应的数据

## 基本功能

### 重建索引

`reindex`是pandas的重要方法，用于**创建一个符合新索引的新对象**


参数 | 描述
- | - 
index |新建作为索引的序列，可以是索引实例或任意其他序列型Python数据结构，索引是使用时无需复制
method | 插值方式， `ffill` 前向插值  `bfill` 后向插值
fill_value | 通过重新索引引入缺失数据时的替代值
limit  | 当前向或后向时，所需填充的最大尺度间隙（元素数量）
tolerance | 当前向或后向时，所需填充的不精确匹配下最大尺度间隙
level | 匹配MultiIndex基本的简单索引，否则选择子集
copy | True 即使旧索引等于新索引，也总是赋值底层的数据， False  索引相同时不要复制数据

In [129]:
obj = Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b','a','c'])
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

In [130]:
obj.reindex(list('abcde'))

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

In [132]:
obj3 = Series(['blue', 'green', 'red'], index=[0,2,4])
obj3

0     blue
2    green
4      red
dtype: object

In [134]:
obj3.reindex( index=range(6), method='ffill' )

0     blue
1     blue
2    green
3    green
4      red
5      red
dtype: object

In [135]:
obj3.reindex( index=range(6), method='bfill' )

0     blue
1    green
2    green
3      red
4      red
5      NaN
dtype: object

In [139]:
frame = DataFrame( np.arange(9).reshape((3,3)), index=['a', 'c', 'd'], columns=['Ohio', 'Texas', 'California'] )
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [140]:
frame.reindex(list('abcd'))

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


In [143]:
frame.reindex(columns=['Texas', 'Utah', 'California'],index=list('abcd') )

,Texas,Utah,California
a,1.0,NaN,2.0
b,NaN,NaN,NaN
c,4.0,NaN,5.0
d,7.0,NaN,8.0


In [146]:
frame.loc[ ['a', 'b', 'c', 'd'], ['Texas', 'Utah', 'California'] ]

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Index(['b'], dtype='object'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [149]:
frame.loc[ ['a', 'c', 'd' , 'b']]

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Index(['b'], dtype='object'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

### 轴向上删除条目

拥有索引数组或不含条目的列表，**在轴向上删除一个或更多的条目**就非常容易

`drop`方法会**返回一个含有指示值或轴向上删除值的新对象**
- `inplace`:参数：表示**在原对象上操作**
- `Series`:一个直接传入，多个当做列表传入

In [150]:
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

In [151]:
obj.drop('c')

d    4.5
b    7.2
a   -5.3
dtype: float64

In [152]:
obj # 

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

In [154]:
obj.drop(['a', 'b'], inplace=True)

In [155]:
obj 

d    4.5
c    3.6
dtype: float64

In [156]:
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [157]:
frame.drop(['c', 'a'])

,Ohio,Texas,California
d,6,7,8


In [159]:
frame.drop(columns=['Ohio','California'])

,Texas
a,1
c,4
d,7


In [161]:
frame.drop(index=['a', 'c'],columns=['Ohio','California'])

,Texas
d,7


### 索引，选择和过滤

Series的`[obj[...]]`与NumPy数组索引功能类似

- Series的索引值可**不仅仅是整数**
- Series的**索引切片包含尾部的**，但整数索引不包括

In [163]:
obj = Series(np.arange(4.), index=list('abcd'))
obj

a    0.0
b    1.0
c    2.0
d    3.0
dtype: float64

In [164]:
obj['a']

0.0

In [165]:
obj['a':'c'] # 索引切片包含尾部

a    0.0
b    1.0
c    2.0
dtype: float64

In [167]:
obj[:3]

a    0.0
b    1.0
c    2.0
dtype: float64

In [168]:
obj2 = Series(np.arange(4.))
obj2

0    0.0
1    1.0
2    2.0
3    3.0
dtype: float64

In [170]:
obj2[1:2]

1    1.0
dtype: float64

In [171]:
obj[['a', 'd', 'b']]

a    0.0
d    3.0
b    1.0
dtype: float64

In [173]:
obj > 1

a    False
b    False
c     True
d     True
dtype: bool

In [174]:
obj[ obj > 1 ]

c    2.0
d    3.0
dtype: float64

In [176]:
obj[ obj > 1 ] = 100
obj 

a      0.0
b      1.0
c    100.0
d    100.0
dtype: float64

使用单个值或者序列，**从DataFrame中索引出一个或多个列**

In [177]:
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [178]:
frame = DataFrame( np.arange(9).reshape((3,3)), index=['a', 'c', 'd'], columns=['Ohio', 'Texas', 'California'] )
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [179]:
frame['Ohio']

a    0
c    3
d    6
Name: Ohio, dtype: int64

In [181]:
frame[ ['Ohio', 'Texas']]

,Ohio,Texas
a,0,1
c,3,4
d,6,7


In [191]:
frame['Ohio':'California']

,Ohio,Texas,California


In [185]:
frame[ ['Ohio', 'California']]

,Ohio,California
a,0,2
c,3,5
d,6,8


In [187]:
frame[:1]

,Ohio,Texas,California
a,0,1,2


In [188]:
frame ['Ohio'] > 2

a    False
c     True
d     True
Name: Ohio, dtype: bool

In [192]:
frame > 3

,Ohio,Texas,California
a,False,False,False
c,False,True,True
d,True,True,True


In [193]:
frame [ frame > 3 ]

,Ohio,Texas,California
a,NaN,NaN,NaN
c,NaN,4.0,5.0
d,6.0,7.0,8.0


In [195]:
frame [ frame > 3 ] = 0
frame

,Ohio,Texas,California
a,0,1,2
c,3,0,0
d,0,0,0


#### 使用iloc和loc选择数据

针对DataFrame在行上的标签索引，使用特殊的`loc`和`iloc`

允许使用轴标签(loc)或整数标签(iloc以NumPy风格的语法从DataFrame中选出数组和行和列的子集

In [196]:
frame = DataFrame( np.arange(9).reshape((3,3)), index=['a', 'c', 'd'], columns=['Ohio', 'Texas', 'California'] )
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [199]:
frame.loc['a' ]

Ohio          0
Texas         1
California    2
Name: a, dtype: int64

In [202]:
frame.loc['a', ['Ohio','Texas']] #

Ohio     0
Texas    1
Name: a, dtype: int64

In [203]:
frame.loc[['a', 'c'], ['Ohio','Texas']] #

,Ohio,Texas
a,0,1
c,3,4


In [204]:
frame.iloc[2]

Ohio          6
Texas         7
California    8
Name: d, dtype: int64

In [205]:
frame.iloc[1:2, 1:3]

,Texas,California
c,4,5


In [206]:
frame.loc[ 'a':'c', 'Ohio':'Texas' ]

,Ohio,Texas
a,0,1
c,3,4


In [213]:
frame.iloc[: , :3]

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [214]:
[frame['Ohio'] > 2]

[a    False
 c     True
 d     True
 Name: Ohio, dtype: bool]

In [215]:
frame.iloc[: , :3][frame['Ohio'] > 2]

,Ohio,Texas,California
c,3,4,5
d,6,7,8


DataFrame 索引选项

类型 | 描述
- | -
df[val] | 从DataFrame中选取单列或者列序列：特殊情况的便利：布尔数组(过滤行)，切片(切片行)或布尔值DataFrame
df.loc[val] | 根据标签选择DataFrame单行或多行
df.loc[:, val]| 根据标签选择单列或多列
df.loc[val1, val2]| 同时选择行和列中的一部分
df.iloc[where]| 根据整数位数选择单行或多行
df.iloc[:, where] | 根据整数位数选择单列或多列
df.iloc[where1, where2] | 同时选择行和列中的一部分
df.at[label_i ,balel_j] | 根据行列标签选择单个标量值
df.iat[i, j] | 根据行列整数选择单个标量值
reindex | 通过标签选择行和列
get_value, set_value | 根据行列的标签设值单个值

### 整数索引

在`pandas`对象**用整数索引经常产生歧义**，因为它和在列表，元组等Python内建数据结构上进行索引有些许不同

In [224]:
ser = Series(np.arange(3.))
ser

0    0.0
1    1.0
2    2.0
dtype: float64

In [225]:
ser.loc[:1]

0    0.0
1    1.0
dtype: float64

In [217]:
ser[-1]

KeyError: -1

In [218]:
ser = Series(np.arange(3.), index=list('abc'))
ser

a    0.0
b    1.0
c    2.0
dtype: float64

In [219]:
ser[-1]

2.0

为了保值一致性，**如果有一个包含整数的轴索引，数据选择时请始终使用标签索引**

**为了更精确的处理，可以使用loc(用于标签)或iloc（用于整数）**
- `loc`： 包含标签的尾部
- `iloc`: 整数，不包含尾部

In [220]:
ser.loc

In [222]:
ser.iloc[ :1]

a    0.0
dtype: float64

### 算术和数据对齐

不同索引的对象之间的算术行为是`pandas`提供给一些应用的一项重要特性

当你将对象相加时，如果存在某个索引对不相同，则**返回结果将是索引对的并集**，类似数据库中的索引标签的**自动外连接**（`outer join`）

**没有交叠的标签位置上，内部数据对齐会产生缺失值**

In [228]:
s1 = Series([7.3, -2.4, 3.4, 1.5], index=['a', 'c', 'd', 'e'])
s2 = Series( [-2.4, 3.6, -1.5, 4,3.1], index=list('acefg') )
s1

a    7.3
c   -2.4
d    3.4
e    1.5
dtype: float64

In [229]:
s2 

a   -2.4
c    3.6
e   -1.5
f    4.0
g    3.1
dtype: float64

In [230]:
s1 + s2 

a    4.9
c    1.2
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

`DataFrame`的算术运算中，**行和列都会执行对齐**,返回的索引，列是每个`DataFrame`的索引，列的**并集**
- 将两个列或行完全不同的`DataFrame`相加，结果将**全部为空**

In [234]:
df1 = DataFrame(np.arange(9.).reshape((3,3)),columns=list('bcd'), index=['Ohio', 'Texas', 'Colorado'])
df2 = DataFrame(np.arange(12.).reshape((4,3)),columns=list('bde'), index=['Utah','Ohio', 'Texas', 'Oregon'])
df1

,b,c,d
Ohio,0.0,1.0,2.0
Texas,3.0,4.0,5.0
Colorado,6.0,7.0,8.0


In [235]:
df2

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [236]:
df1 + df2 

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
Ohio,3.0,NaN,6.0,NaN
Oregon,NaN,NaN,NaN,NaN
Texas,9.0,NaN,12.0,NaN
Utah,NaN,NaN,NaN,NaN


In [237]:
df11 = DataFrame({"A": [1,2]})
df12 = DataFrame({"B": [1,2]})
df11

,A
0,1
1,2


In [238]:
df12

,B
0,1
1,2


In [240]:
df11 + df12

,A,B
0,NaN,NaN
1,NaN,NaN


#### 使用填充值的算术方法

在两个不同索引对象之间进行算术运算时，可能会**想要使用特殊填充值**， 当轴标签在一个对象中存在，在另一个对象中不存在时，想将缺失值填充为0

这时，就**需要使用函数，或者方法，可以传入参数`fill_value`**

In [241]:
df1 = DataFrame(np.arange(12.).reshape((3,4)), columns=list('abcd'))
df2 = DataFrame(np.arange(20.).reshape((4,5)), columns=list('abcde'))
df1 

,a,b,c,d
0,0.0,1.0,2.0,3.0
1,4.0,5.0,6.0,7.0
2,8.0,9.0,10.0,11.0


In [242]:
df2

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.0,8.0,9.0
2,10.0,11.0,12.0,13.0,14.0
3,15.0,16.0,17.0,18.0,19.0


In [243]:
df2.loc[1, 'b'] =np.nan

In [244]:
df2

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,4.0
1,5.0,NaN,7.0,8.0,9.0
2,10.0,11.0,12.0,13.0,14.0
3,15.0,16.0,17.0,18.0,19.0


In [245]:
df1 + df2

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,NaN
1,9.0,NaN,13.0,15.0,NaN
2,18.0,20.0,22.0,24.0,NaN
3,NaN,NaN,NaN,NaN,NaN


In [246]:
df1.add(df2, fill_value=0)

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,4.0
1,9.0,5.0,13.0,15.0,9.0
2,18.0,20.0,22.0,24.0,14.0
3,15.0,16.0,17.0,18.0,19.0


In [247]:
df2.add(df1, fill_value=0)

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,4.0
1,9.0,5.0,13.0,15.0,9.0
2,18.0,20.0,22.0,24.0,14.0
3,15.0,16.0,17.0,18.0,19.0


`fill_value`:表示运算时，将缺失值替换为`fill_value`，再参与运算

当Series或者DataFrame重新索引时，可以指定一个不同的**填充值**

In [250]:
df1.loc[1,'b'] =np.nan

In [251]:
df2.add(df1, fill_value=0)

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,4.0
1,9.0,NaN,13.0,15.0,9.0
2,18.0,20.0,22.0,24.0,14.0
3,15.0,16.0,17.0,18.0,19.0


两个DataFrame对应的位置都是`NA`，则结果依然是`NA`

#### DataFrame与Series之间的操作

`Series`和`DataFrame`间的算术操作与`NumPy`中的**不同维度数组**间的操作类型

In [252]:
arr = np.arange(12.).reshape((3,4))
arr

array([[ 0.,  1.,  2.,  3.],
       [ 4.,  5.,  6.,  7.],
       [ 8.,  9., 10., 11.]])

In [253]:
arr[0]

array([0., 1., 2., 3.])

In [255]:
arr - arr[0]

array([[0., 0., 0., 0.],
       [4., 4., 4., 4.],
       [8., 8., 8., 8.]])

In [256]:
frame = DataFrame(np.arange(12.).reshape((4,3)), columns=list('bde'), index=['Utah', 'Ohio',' Texas', 'Oregon'])
frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [257]:
ser = frame.iloc[0]
ser 

b    0.0
d    1.0
e    2.0
Name: Utah, dtype: float64

In [260]:
ser.shape # 需要补0轴

(3,)

In [261]:
frame - ser

,b,d,e
Utah,0.0,0.0,0.0
Ohio,3.0,3.0,3.0
Texas,6.0,6.0,6.0
Oregon,9.0,9.0,9.0


默认情况下，`DataFrame`和`Series`的数学操作中将`Series`的索引和`DataFrame`的**列进行匹配**，**并广播到各行**

shape扩展
- 将shape中的一个数当做index或column
- 默认当做column `(3,)--->(1,3)-->(3,3)` 行扩展
- index参数，当做index `(3,)--->(3,1)-->(3,3)` 列扩展

In [259]:
frame.sub??
# axis : {0 or 'index', 1 or 'columns'  
# Whether to compare by the index (0 or 'index') or columns
# b(1 or 'columns'). For Series input, axis to match Series index on.

`axis`参数 用于**匹配轴**

![广播](https://gitee.com/zhao_long/image-store/raw/master/img/20201113094932.png)

### 函数应用和映射

NumPy的通用函数(逐元素数组方法)对`pandas`对象也有效

In [263]:
frame = DataFrame(np.random.randn(4,3), columns=list('bde'), index=['Utah', 'Ohio',' Texas', 'Oregon'])
frame

,b,d,e
Utah,-0.204708,0.478943,-0.519439
Ohio,-0.555730,1.965781,1.393406
Texas,0.092908,0.281746,0.769023
Oregon,1.246435,1.007189,-1.296221


In [264]:
frame.apply?? #axis=0, raw=False, result_type=None, 

 {0 or 'index', 1 or 'columns'}, default 0
            Axis along which the function is applied:

            * 0 or 'index': apply function to each column.
            * 1 or 'columns': apply function to each row.


In [265]:
np.abs(frame)

,b,d,e
Utah,0.204708,0.478943,0.519439
Ohio,0.555730,1.965781,1.393406
Texas,0.092908,0.281746,0.769023
Oregon,1.246435,1.007189,1.296221


将函数应用到一行或者一列的一维数组上，`DataFrame`的`apply`方法可以实现

In [266]:
f = lambda x: x.max() - x.min()

In [267]:
frame.apply(f)

b    1.802165
d    1.684034
e    2.689627
dtype: float64

In [268]:
frame.apply(f, axis=1)

Utah      0.998382
Ohio      2.521511
 Texas    0.676115
Oregon    2.542656
dtype: float64

In [273]:
frame.apply(f)

,b,d,e
amx,1.246435,1.965781,1.393406
min,-0.555730,0.281746,-1.296221


In [272]:
f = lambda x: Series([x.max(), x.min()], index=['amx', 'min'])

In [274]:
frame.apply(f,axis=1)

,amx,min
Utah,0.478943,-0.519439
Ohio,1.965781,-0.555730
Texas,0.769023,0.092908
Oregon,1.246435,-1.296221


`applymap`在pandas中表示 **逐元素**

In [275]:
f = lambda x: "%0.2f" %x

In [276]:
frame

,b,d,e
Utah,-0.204708,0.478943,-0.519439
Ohio,-0.555730,1.965781,1.393406
Texas,0.092908,0.281746,0.769023
Oregon,1.246435,1.007189,-1.296221


In [278]:
frame.applymap(f)

,b,d,e
Utah,-0.20,0.48,-0.52
Ohio,-0.56,1.97,1.39
Texas,0.09,0.28,0.77
Oregon,1.25,1.01,-1.30


### 排序和排名

根据某些准则对数据进行排序是另一个重要的内建操作

如需**对行或列进行字典型排序**，需要使用`sort_index`方法，该方法返回一个新的，排序好的对象

In [279]:
obj = Series(range(4), index=list('dabc'))
obj 

d    0
a    1
b    2
c    3
dtype: int64

In [280]:
obj.sort_index()

a    1
b    2
c    3
d    0
dtype: int64

In [281]:
frame  = DataFrame( np.arange(8).reshape((2, 4)), index=['three', 'one'], columns=list('dabc') )
frame

,d,a,b,c
three,0,1,2,3
one,4,5,6,7


In [282]:
frame.sort_index()

,d,a,b,c
one,4,5,6,7
three,0,1,2,3


In [283]:
frame.sort_index(axis=1)

,a,b,c,d
three,1,2,3,0
one,5,6,7,4


In [284]:
frame.sort_index(axis=1, ascending=False)

,d,c,b,a
three,0,3,2,1
one,4,7,6,5


`sort_value`实现**对值的排序**， **使用一列或多列作为排序键** 参数`by`:必须传值

In [285]:
obj

d    0
a    1
b    2
c    3
dtype: int64

In [286]:
obj.sort_values()

d    0
a    1
b    2
c    3
dtype: int64

In [287]:
frame

,d,a,b,c
three,0,1,2,3
one,4,5,6,7


In [289]:
frame.sort_values(by='a')

,d,a,b,c
three,0,1,2,3
one,4,5,6,7


`rank`： 对数组**从1到有效数据点总数**分配名词的操作

方法 | 描述
- | - 
average | 默认，在**每组**中分配**平均排名**
min | 对整个组使用最小排名
max | 对整个组使用最大排名
first | 按照值在数据中出现的次序分配排名
dense | 类似`method=min`， 但是**组间排名总是增加1**，而不是一个组中的相等元素的数量

In [290]:
obj = Series([7, -5, 7, 4,2,0,4])
obj

0    7
1   -5
2    7
3    4
4    2
5    0
6    4
dtype: int64

In [291]:
obj.rank()

0    6.5
1    1.0
2    6.5
3    4.5
4    3.0
5    2.0
6    4.5
dtype: float64

In [292]:
obj.rank(method='dense')

0    5.0
1    1.0
2    5.0
3    4.0
4    3.0
5    2.0
6    4.0
dtype: float64

In [293]:
obj.rank(method='first')

0    6.0
1    1.0
2    7.0
3    4.0
4    3.0
5    2.0
6    5.0
dtype: float64

In [294]:
obj.rank(method='max')

0    7.0
1    1.0
2    7.0
3    5.0
4    3.0
5    2.0
6    5.0
dtype: float64

In [295]:
obj.rank(method='min')

0    6.0
1    1.0
2    6.0
3    4.0
4    3.0
5    2.0
6    4.0
dtype: float64

In [296]:
obj.rank(method='min', ascending=False)

0    1.0
1    7.0
2    1.0
3    3.0
4    5.0
5    6.0
6    3.0
dtype: float64

In [297]:
frame

,d,a,b,c
three,0,1,2,3
one,4,5,6,7


In [298]:
frame.rank()

,d,a,b,c
three,1.0,1.0,1.0,1.0
one,2.0,2.0,2.0,2.0


In [299]:
frame.rank(axis=1)

,d,a,b,c
three,1.0,2.0,3.0,4.0
one,1.0,2.0,3.0,4.0


### 含有重复标签的轴索引

尽管很多的`pandas`函数需要标签是唯一的，但是并不强制

带有重复索引的情况下，数据选择和之前操作有差别，**根据一个标签索引多个条目会返回一个序列，单个条目返回变量值**

`is_unique`属性返回索引是否唯一

In [301]:
obj = Series(range(5), index=list('aabbc'))
obj

a    0
a    1
b    2
b    3
c    4
dtype: int64

In [303]:
obj.index.is_unique

False

In [304]:
obj['a']

a    0
a    1
dtype: int64

In [306]:
df = DataFrame(np.random.randn(4,3), index=list('aabb'))
df

,0,1,2
a,0.274992,0.228913,1.352917
a,0.886429,-2.001637,-0.371843
b,1.669025,-0.438570,-0.539741
b,0.476985,3.248944,-1.021228


In [307]:
df.loc['b']

,0,1,2
b,1.669025,-0.438570,-0.539741
b,0.476985,3.248944,-1.021228


## 描述性统计的概述和计算

`pandas`对象装配了一个常用数学，统计学方法的集合，其中大部分属于**归约或汇总**统计的类别，这些方法**从`DataFrame`的行或列抽取一个Series或一系列单个值**，与NumPy数组的类似方法相比，**内建了处理缺失值的功能**

In [309]:
df  = DataFrame( [ [1.4, np.nan] , [7.1, -4.5] , [np.nan, np.nan], [0.75, -1.3]], index=['a', 'b', 'c', 'd'], columns=['one', 'two'] )
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


In [310]:
df.sum()

one    9.25
two   -5.80
dtype: float64

In [311]:
df.sum(1)

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

In [312]:
df.sum(1, skipna=False)

a     NaN
b    2.60
c     NaN
d   -0.55
dtype: float64

In [316]:
df.idxmax()

one    b
two    d
dtype: object

In [317]:
df.argmax?

Object `df.argmax` not found.


In [318]:
df.cumsum()

,one,two
a,1.40,NaN
b,8.50,-4.5
c,NaN,NaN
d,9.25,-5.8


In [319]:
df.cummax()

,one,two
a,1.4,NaN
b,7.1,-4.5
c,NaN,NaN
d,7.1,-1.3


In [320]:
df.describe()

,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,1.075000,-3.700000
50%,1.400000,-2.900000
75%,4.250000,-2.100000
max,7.100000,-1.300000


In [321]:
obj = Series(['a', 'a', 'b', 'c'] * 4)
obj

0     a
1     a
2     b
3     c
4     a
5     a
6     b
7     c
8     a
9     a
10    b
11    c
12    a
13    a
14    b
15    c
dtype: object

In [322]:
obj.describe()

count     16
unique     3
top        a
freq       8
dtype: object

### 相关性和协方差

一些汇总统计，比如相关性和协方差，是**由多个参数计算出来的**

In [323]:
prices = pd.read_pickle('pydata-book/examples/yahoo_price.pkl')
volume = pd.read_pickle('pydata-book/examples/yahoo_volume.pkl')

In [324]:
prices # 收盘价

,AAPL,GOOG,IBM,MSFT
Date,,,,
2010-01-04,27.990226,313.062468,113.304536,25.884104
2010-01-05,28.038618,311.683844,111.935822,25.892466
2010-01-06,27.592626,303.826685,111.208683,25.733566
2010-01-07,27.541619,296.753749,110.823732,25.465944
2010-01-08,27.724725,300.709808,111.935822,25.641571
...,...,...,...,...
2016-10-17,117.550003,779.960022,154.770004,57.220001
2016-10-18,117.470001,795.260010,150.720001,57.660000
2016-10-19,117.120003,801.500000,151.259995,57.529999


In [325]:
volume# 成交量

,AAPL,GOOG,IBM,MSFT
Date,,,,
2010-01-04,123432400,3927000,6155300,38409100
2010-01-05,150476200,6031900,6841400,49749600
2010-01-06,138040000,7987100,5605300,58182400
2010-01-07,119282800,12876600,5840600,50559700
2010-01-08,111902700,9483900,4197200,51197400
...,...,...,...,...
2016-10-17,23624900,1089500,5890400,23830000
2016-10-18,24553500,1995600,12770600,19149500
2016-10-19,20034600,116600,4632900,22878400


In [327]:
pip install pandas-datareader

Looking in indexes: https://pypi.doubanio.com/simple
     |████████████████████████████████| 107 kB 160 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [330]:
import  pandas_datareader.data as web

all_data = {
    ticker: web.get_data_yahoo(ticker) for ticker in ['AAPL', 'IBM', 'MSFT', 'GOOG']
}

ConnectionError: HTTPSConnectionPool(host='finance.yahoo.com', port=443): Read timed out.

In [ ]:
all_data

In [332]:
result  = prices.pct_change()

In [333]:
result.head()

,AAPL,GOOG,IBM,MSFT
Date,,,,
2010-01-04,NaN,NaN,NaN,NaN
2010-01-05,0.001729,-0.004404,-0.012080,0.000323
2010-01-06,-0.015906,-0.025209,-0.006496,-0.006137
2010-01-07,-0.001849,-0.023280,-0.003462,-0.010400
2010-01-08,0.006648,0.013331,0.010035,0.006897


In [334]:
result.tail()

,AAPL,GOOG,IBM,MSFT
Date,,,,
2016-10-17,-0.000680,0.001837,0.002072,-0.003483
2016-10-18,-0.000681,0.019616,-0.026168,0.007690
2016-10-19,-0.002979,0.007846,0.003583,-0.002255
2016-10-20,-0.000512,-0.005652,0.001719,-0.004867
2016-10-21,-0.003930,0.003011,-0.012474,0.042096


In [337]:
result['MSFT'].corr(result['GOOG'])

0.4659187949079665

In [338]:
result['MSFT'].corr(result['IBM'])

0.49976361144151144

In [339]:
result['MSFT'].cov(result['IBM'])

8.870655479703546e-05

In [340]:
result.cov()

,AAPL,GOOG,IBM,MSFT
AAPL,0.000277,0.000107,0.000078,0.000095
GOOG,0.000107,0.000251,0.000078,0.000108
IBM,0.000078,0.000078,0.000146,0.000089
MSFT,0.000095,0.000108,0.000089,0.000215


In [341]:
result.corr()

,AAPL,GOOG,IBM,MSFT
AAPL,1.000000,0.407919,0.386817,0.389695
GOOG,0.407919,1.000000,0.405099,0.465919
IBM,0.386817,0.405099,1.000000,0.499764
MSFT,0.389695,0.465919,0.499764,1.000000


In [342]:
result.corrwith(result.IBM)

AAPL    0.386817
GOOG    0.405099
IBM     1.000000
MSFT    0.499764
dtype: float64

In [343]:
result.corrwith(result)

AAPL    1.0
GOOG    1.0
IBM     1.0
MSFT    1.0
dtype: float64

### 唯一性，计数 和 成员属性

另一类相关的方法可以**从一维度`Series`包含的数值中提取信息**

- `unique`: 给出Series中的唯一值
- `value_counts`: 计算Series包含值的个数，返回**的Series按照数量降序排序，索引时unique的Series**, 也是有效的pandas函数

In [344]:
obj = Series(list('cadaabbbcc'))
obj

0    c
1    a
2    d
3    a
4    a
5    b
6    b
7    b
8    c
9    c
dtype: object

In [346]:
obj.unique()

array(['c', 'a', 'd', 'b'], dtype=object)

In [348]:
obj.value_counts()

a    3
c    3
b    3
d    1
dtype: int64

In [351]:
pd.value_counts(obj.values, sort=True)

a    3
c    3
b    3
d    1
dtype: int64

`isin`执行**向量的成员属性检查**，还可以将数据集以Series或DataFrame一列的形式过滤为数据集的子集

In [352]:
obj

0    c
1    a
2    d
3    a
4    a
5    b
6    b
7    b
8    c
9    c
dtype: object

In [353]:
'c' in obj

False

In [354]:
1 in obj

True

In [356]:
obj.isin([1])

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
9    False
dtype: bool

In [357]:
obj.isin(['a', 'b'])

0    False
1     True
2    False
3     True
4     True
5     True
6     True
7     True
8    False
9    False
dtype: bool

In [358]:
obj[obj.isin(['a', 'b'])]

1    a
3    a
4    a
5    b
6    b
7    b
dtype: object

`Index.get_indexer`方法**获取一个索引数组**，从可能非唯一值数组转化为另一个唯一值数组

In [359]:
to_match = Series(list('cabbca'))

In [360]:
to_match

0    c
1    a
2    b
3    b
4    c
5    a
dtype: object

In [361]:
unique_vals = Series(['c' , 'b' ,'d'])

In [362]:
unique_vals

0    c
1    b
2    d
dtype: object

In [363]:
pd.Index(unique_vals).get_indexer(to_match)

array([ 0, -1,  1,  1,  0, -1])

In [364]:
pd.Index(unique_vals)

Index(['c', 'b', 'd'], dtype='object')

In [365]:
data = DataFrame({
    'Qu1':[1,3,4,3,4],
    'Qu2':[2,3,1,2,3],
    'Qu3':[1,5,2,4,4,]
})
data 

,Qu1,Qu2,Qu3
0,1,2,1
1,3,3,5
2,4,1,2
3,3,2,4
4,4,3,4


In [367]:
data.apply(pd.value_counts).fillna(0)

,Qu1,Qu2,Qu3
1,1.0,1.0,1.0
2,0.0,2.0,1.0
3,2.0,2.0,0.0
4,2.0,0.0,2.0
5,0.0,0.0,1.0
